# MAIN
1. split main task into stages
2. epoch motor planning (-500, 500) around target onset
3. epoch motor execution (-500, 700) around movement onset
4. time-frequency for individual subjects + group analysis


In [ ]:
import mne
import os
from utils import check_paths
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
%matplotlib qt

# PLANNING / EXECUTION

1. EPOCHING

In [43]:
print(os.listdir(subs_dir))

['s1_pac_sub12', 's1_pac_sub14', 's1_pac_sub15', 's1_pac_sub17', 's1_pac_sub20', 's1_pac_sub21', 's1_pac_sub23', 's1_pac_sub34', 's1_pac_sub35', 's1_pac_sub36', 's1_pac_sub40', 's1_pac_sub41', 's1_pac_sub45', 's1_pac_sub47', 's1_pac_sub48', 's1_pac_sub49', 's1_pac_sub50', 's1_pac_sub53', 's1_pac_sub54', 's1_pac_sub55', 's1_pac_sub56', 's1_pac_sub60', 's1_pac_sub62', 's1_pac_sub68']


In [ ]:
# Create planning / execution epochs for subjects
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group = 'O'
task = '_MAIN' # ['_BL', '_MAIN']
task_stage = '_plan' # '_plan' for PLANNING or '_go' for EXECUTION

subs_dir = os.path.join(eeg_data_dir, group)
figs_dir = os.path.join(eeg_data_dir, 'figures', group, 'epochs', task)
check_paths(figs_dir)

# trial infor dictionary
trial_num_data = {'sub': [], 'task_stage': [], 'block_name': [], 'trial_num': []}

# metadata for each epoch shall include events from the range: [-0.5, 0.7] s,
# i.e. starting with stimulus onset and expanding beyond the end of the epoch
if task_stage == '_plan':
    metadata_tmin, metadata_tmax = -0.5, 0.5 # [-0.5, 0.7] for _go or [-0.5, 0.5] for _plan
    # events of interest
    row_events = ['target_on'] # ['target_on'] or ['go_on']
    # timing of the epochs
    epochs_tmin, epochs_tmax = -0.5, 0.5  # epochs range: [-0.5, 0.7] for _go or [-0.5, 0.5] for _plan

else:
    metadata_tmin, metadata_tmax = -0.5, 0.7 # [-0.5, 0.7] for _go or [-0.5, 0.5] for _plan
    # events of interest
    row_events = ['go_on'] # ['target_on'] or ['go_on']
    # timing of the epochs
    epochs_tmin, epochs_tmax = -0.5, 0.7  # epochs range: [-0.5, 0.7] for _go or [-0.5, 0.5] for _plan

baseline_epo = (epochs_tmin, epochs_tmax) # baseline is the average of the epoch


for sub_name in os.listdir(subs_dir): # os.listdir(subs_dir) OR ['s1_pac_sub00'] # EXCLUDED sub19 - no baseline trigger

    print(f'Creating epochs for {task} task in {sub_name}...')

    preproc_dir = os.path.join(subs_dir, sub_name, 'preproc')
    filt_dir = os.path.join(subs_dir, sub_name, 'preproc', 'filt')
    analysis_dir = os.path.join(preproc_dir, 'analysis')

    eeg_data_path = os.path.join(analysis_dir, f'{sub_name}{task}_reconst.fif')
    raw = mne.io.read_raw_fif(eeg_data_path, preload=True)
    sf = raw.info['sfreq'] # sampling frequency of data

    # Open events from pickle file
    with open(os.path.join(filt_dir, f'{sub_name}{task}_events.pkl'), 'rb') as pickle_file:
        events_raw = pickle.load(pickle_file)

    # check if event time samples are unique
    for i in range(len(events_raw[0]) - 1):
        if events_raw[0][i, 0] == events_raw[0][i+1, 0]:
            print(f'Time sample for events_raw {events_raw[0][i, 2]} and {events_raw[0][i+1, 2]} in trials {i}-{i+1} are not unique!')
            events_raw[0][i+1, 0] = events_raw[0][i, 0] + 1


    # extract time stamps (in samples) from the events_raw[0] array for key in events_raw[1] dict
    bl_start_sample = events_raw[0][events_raw[0][:, 2] == events_raw[1]['baseline']] 
    adapt_start_sample = events_raw[0][events_raw[0][:, 2] == events_raw[1]['adapt1']]
    adapt_finish_sample = events_raw[0][events_raw[0][:, 2] == events_raw[1]['postadapt']]
    
    # convert time stamps from sample to seconds
    bl_start_time = bl_start_sample[0, 0] / sf
    adapt_start_time = adapt_start_sample[0, 0] / sf
    adapt_finish_time = adapt_finish_sample[0, 0] / sf

    print(f'''baseline block: {bl_start_time}-{adapt_start_time} sec ~ {(adapt_start_time-bl_start_time)/60:.2f} min
    adaptation block: {adapt_start_time}-{adapt_finish_time} sec ~ {(adapt_finish_time-adapt_start_time)/60:.2f} min''')

    # crop raw file into segments: baseline block and adaptation block
    bl_raw = raw.copy().crop(tmin=(bl_start_time-1), tmax=(adapt_start_time+1)) # add and subtract 1 to catch start and end events
    adapt_raw = raw.copy().crop(tmin=(adapt_start_time-1), tmax=(adapt_finish_time+1))

    for block_raw in [bl_raw, adapt_raw]:

        if block_raw == bl_raw:
            block_name = '_baseline'
        else:
            block_name = '_adaptation'

        # auto-create metadata:
        # this also returns a new events array and an event_id dictionary. we'll see
        # later why this is important
        metadata, events, event_id = mne.epochs.make_metadata(
            events=events_raw[0],
            event_id=events_raw[1],
            tmin=metadata_tmin,
            tmax=metadata_tmax,
            sfreq=sf,
            row_events=row_events
        )

        epochs = mne.Epochs(
            raw=block_raw,
            tmin=epochs_tmin,
            tmax=epochs_tmax,
            events=events,
            event_id=event_id,
            baseline=baseline_epo,
            detrend=None,
            metadata=metadata,
            reject_by_annotation=True,
            preload=True,
        )

        if task_stage == '_plan':
            epochs = epochs["trial_start.isna() & go_on.isna() & bad_early.isna()"]
        
        else:
            if 'bad_late' in events_raw[1]:
                epochs = epochs["trial_start.isna() & bad_early.isna() & bad_late.isna()"]
            else:
                epochs = epochs["trial_start.isna() & bad_early.isna()"]

        print(f'TOTAL NUMBER OF EPOCHS: {len(epochs)}')

        # Append data to the dictionary
        trial_num_data['sub'].append(sub_name)
        trial_num_data['task_stage'].append(task_stage)
        trial_num_data['block_name'].append(block_name)
        trial_num_data['trial_num'].append(len(epochs))

        # Save the epochs
        epochs.save(os.path.join(analysis_dir, f"{sub_name}{task}_epochs{task_stage}{block_name}-epo.fif"), overwrite=True)
        print(f'Epochs for {task}_{block_name} in {sub_name} saved SUCCESSFULLY')

        # Plot ERP
        fig_erp = epochs.average().plot(gfp=True, spatial_colors=True)
        # Save the ERP plot
        fig_erp.savefig(os.path.join(figs_dir, f"{sub_name}{task}{task_stage}{block_name}_erp_plot.png"), dpi=300)


        spectrum = epochs.compute_psd()
        bands = {'Theta (4-8 Hz)': (4, 8),
                'Alpha (8-12 Hz)': (8, 12),
                'Beta (12-30 Hz)': (12, 30),
                'Gamma (30-50 Hz)': (30, 50),
                'High gamma (50-80 Hz)': (50, 80)}

        fig_psd = spectrum.plot_topomap(bands=bands, vlim="joint", normalize=True)
        fig_psd.savefig(os.path.join(figs_dir, f"{sub_name}{task}{task_stage}{block_name}_psd_topomap.png"), dpi=300)

        print(f'Figures for {sub_name} saved SUCCESSFULLY')

        plt.close('all')

#save the trial number data to a CSV file
trial_num_df = pd.DataFrame(trial_num_data)
trial_num_df.to_csv(os.path.join(figs_dir, f"{task[1:]}{task_stage}_TRIAL_NUM.csv"), index=False)
print(f'TRIAL NUMBER saved SUCCESSFULLY to {os.path.join(figs_dir, f"{task[1:]}{task_stage}_TRIAL_NUM.csv")}')

2. Time-frequency

In [ ]:
# Run time-frequency analysis for individual subjects
# task_stage = '_go' # '_plan' or '_go'
baseline_tf = (-0.5, -0.1) # baseline for TF is prior to the event of interest (zscore)

for sub_name in os.listdir(subs_dir): # os.listdir(subs_dir) OR ['s1_pac_sub00']

    print(f'Creating TF maps for {task} task in {sub_name}...')

    preproc_dir = os.path.join(subs_dir, sub_name, 'preproc')
    analysis_dir = os.path.join(preproc_dir, 'analysis')
    tfr_dir = os.path.join(figs_dir, 'TFR')
    check_paths(tfr_dir)

    for block_name in ['_baseline', '_adaptation']:
        epochs_data_path = os.path.join(analysis_dir, f'{sub_name}{task}_epochs{task_stage}{block_name}-epo.fif')
        epochs = mne.read_epochs(epochs_data_path, preload=True)

        freqs = np.logspace(*np.log10([4, 80]), num=40)
        n_cycles = freqs / 2.0  # different number of cycle per frequency
        power, itc = epochs.compute_tfr(
            method="morlet",
            freqs=freqs,
            n_cycles=n_cycles,
            average=True,
            return_itc=True,
            decim=3,
        )
        power.apply_baseline(baseline=baseline_tf, mode="zscore")

        # Save time-freq analysis output
        power.save(os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_power-tfr.h5"), overwrite=True)
        itc.save(os.path.join(analysis_dir, f"{sub_name}{task}{task_stage}{block_name}_itc-tfr.h5"), overwrite=True)

        # plot ITC and Power
        itc_fig = itc.plot(combine='mean', title=f'Target-reaching task: {task_stage}{block_name}. ITC', cmap='magma')[0]
        power_all_fig = power.plot(mode="zscore", title='Averaged Power Across Channels', combine='mean', cmap='magma')[0]
        power_motor_fig = power.plot(picks=['C1', 'C3', 'C5'], mode="zscore", title='Averaged Power Across C1-5 Channels', combine='mean', cmap='magma')[0]

        # # save ITC and Power plots
        itc_fig.savefig(os.path.join(tfr_dir, f"{sub_name}{task}{task_stage}{block_name}_itc.png"), dpi=300)
        power_all_fig.savefig(os.path.join(tfr_dir, f"{sub_name}{task}{task_stage}{block_name}_power_all.png"), dpi=300)
        power_motor_fig.savefig(os.path.join(tfr_dir, f"{sub_name}{task}{task_stage}{block_name}_power_motor.png"), dpi=300)

        plt.close('all')

TF group

In [69]:
# Concatenate epochs across subjects
# eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
# group = 'Y'
# task = '_MAIN' # ['_BL', '_MAIN']
# task_stage = '_go' # '_plan' or '_go'
block_name = '_adaptation' # ['_baseline', '_adaptation']
# subs_dir = os.path.join(eeg_data_dir, group)

all_epochs = []

for sub_name in os.listdir(subs_dir): # os.listdir(subs_dir) OR ['s1_pac_sub00']

    preproc_dir = os.path.join(subs_dir, sub_name, 'preproc')
    analysis_dir = os.path.join(preproc_dir, 'analysis')

    epochs_data_path = os.path.join(analysis_dir, f'{sub_name}{task}_epochs{task_stage}{block_name}-epo.fif')
    epochs = mne.read_epochs(epochs_data_path, preload=True)
    all_epochs.append(epochs)

epochs_all_subs = mne.concatenate_epochs(all_epochs)

group_save_path = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y group'
epochs_all_subs.save((os.path.join(group_save_path, f"{group}_ALL_subs{task}_epochs{task_stage}{block_name}_ALL-epo.fif")), overwrite=True)

Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub12\preproc\analysis\s1_pac_sub12_MAIN_epochs_plan_adaptation-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 16 columns
135 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub14\preproc\analysis\s1_pac_sub14_MAIN_epochs_plan_adaptation-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 16 columns
143 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 

C:\Users\a1902989\AppData\Local\Temp\ipykernel_26164\3402994877.py:20: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_all_subs = mne.concatenate_epochs(all_epochs)


Adding metadata with 16 columns
3255 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
Overwriting existing file.
Overwriting existing file.


In [70]:
len(all_epochs)

24

In [87]:
group = 'Y'
sub_name = 'ALL_subs'
task = '_MAIN' # ['_BL', '_MAIN']
task_stage = '_go' # '_plan' or '_go'
block_name = '_adaptation' # ['_baseline', '_adaptation']

group_save_path = os.path.join('D:\\BonoKat\\research project\\# study 1\\eeg_data\\set', f'{group} group')

figs_dir = os.path.join('D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures', group, 'epochs', task, 'TFR', block_name)
check_paths(figs_dir)

epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block_name}_ALL-epo.fif"), preload=True)
epochs_all_subs

Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y group\Y_ALL_subs_MAIN_epochs_go_adaptation_ALL-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     700.00 ms
        0 CTF compensation matrices available
Adding metadata with 16 columns
4422 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


<EpochsFIF | 4422 events (all good), -0.5 – 0.7 s (baseline -0.5 – 0.7 s), ~1.21 GB, data loaded, with metadata,
 'go_on': 4422>

In [88]:
freqs = np.logspace(*np.log10([4, 80]), num=40)
n_cycles = freqs / 2.0  # different number of cycle per frequency
baseline_tf = (-0.5, -0.1)

power, itc = epochs_all_subs.compute_tfr(
    method="morlet",
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=True,
    decim=3,
    n_jobs=-1
)
power.apply_baseline(baseline=baseline_tf, mode="zscore")

# Save time-freq analysis output
power.save(os.path.join(group_save_path, f"{group}_{sub_name}{task}{task_stage}{block_name}_power-tfr.h5"), overwrite=True)
itc.save(os.path.join(group_save_path, f"{group}_{sub_name}{task}{task_stage}{block_name}_itc-tfr.h5"), overwrite=True)

# plot ITC and Power
itc_fig = itc.plot(combine='mean', title=f'Group: {group}: Target-reaching task: {task_stage[1:]}{block_name}. ITC', cmap='magma')[0]
power_all_fig = power.plot(mode="zscore", title=f'Group: {group}: Target-reaching task: {task_stage[1:]}{block_name}. Averaged Power', combine='mean', cmap='magma')[0]

# save ITC and Power plots
itc_fig.savefig(os.path.join(figs_dir, f"{group}_{sub_name}{task}{task_stage}{block_name}_itc.png"), dpi=300)
power_all_fig.savefig(os.path.join(figs_dir, f"{group}_{sub_name}{task}{task_stage}{block_name}_power_all.png"), dpi=300)

# plt.close('all')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 out of  60 | elapsed:   23.5s remaining:    2.5s


Applying baseline correction (mode: zscore)


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   24.2s finished


No baseline correction applied
No baseline correction applied


In [40]:
figs_dir

'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures\\Y\\epochs\\_MAIN\\TFR\\_baseline'